In [1]:
 !pip install sentence-transformers==2.2.2 lightgbm scikit-learn pandas numpy tqdm langdetect

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install huggingface_hub==0.11.1


Defaulting to user installation because normal site-packages is not writeable

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.20.3 requires huggingface-hub<1.0,>=0.16.4, but you have huggingface-hub 0.11.1 which is incompatible.
transformers 4.46.2 requires huggingface-hub<1.0,>=0.23.2, but you have huggingface-hub 0.11.1 which is incompatible.



  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.35.3
    Uninstalling huggingface-hub-0.35.3:
      Successfully uninstalled huggingface-hub-0.35.3


In [5]:
!pip install --upgrade sentence-transformers


Defaulting to user installation because normal site-packages is not writeable
  Using cached hf_xet-1.2.0-cp37-abi3-win_amd64.whl.metadata (5.0 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached typer_slim-0.20.0-py3-none-any.whl.metadata (16 kB)
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   ---------------------------------------- 566.1/566.1 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.11.1
    Uninstalling huggingface-hub-0.11.1:
      Successfully uninstalled huggingface-hub-0.11.1
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


In [ ]:

import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from langdetect import detect
import math


DATA_DIR = "./"   
METRIC_NAMES_F = os.path.join(DATA_DIR, "metric_names.json")
METRIC_EMB_F = os.path.join(DATA_DIR, "metric_name_embeddings.npy")   # (145, 768)
TRAIN_F = os.path.join(DATA_DIR, "train_data.json")
TEST_F = os.path.join(DATA_DIR, "test_data.json")
SAMPLE_SUB_F = os.path.join(DATA_DIR, "sample_submission.csv")
OUTPUT_SUB = "submission.csv"
OOF_PRED_F = "oof_preds.csv"


def load_jsonl(path):
    """If JSON lines or list-of-dicts, handle both."""
    with open(path, "r", encoding="utf-8") as f:
        txt = f.read().strip()
        if txt.startswith("["):
            return json.loads(txt)
        else:
            # assume newline separated json objects
            lines = [json.loads(l) for l in txt.splitlines() if l.strip()]
            return lines

def safe_join_text(rec):
    """Concatenate the available text fields into one string for embedding."""
    parts = []
    for k in ["system_prompt", "prompt", "expected_response", "response"]:
        v = rec.get(k) or rec.get(k.lower()) or ""
        if isinstance(v, list):
            v = " ".join(v)
        parts.append(str(v).strip())
    joined = " ||| ".join([p for p in parts if p])
    return joined if joined else " "

def try_lang_detect(text):
    try:
        return detect(text)
    except:
        return "und"


print("Loading dataset files...")
metric_names = json.load(open(METRIC_NAMES_F, "r", encoding="utf-8"))
metric_embeddings = np.load(METRIC_EMB_F) 
print(f"Loaded {len(metric_names)} metric names and embeddings shape {metric_embeddings.shape}")

train = load_jsonl(TRAIN_F)
test = load_jsonl(TEST_F)
sample_sub = pd.read_csv(SAMPLE_SUB_F)

print(f"Train records: {len(train)}, Test records: {len(test)}")


def build_df(records, is_train=True):
    rows = []
    for i, rec in enumerate(records):
        metric_name = rec.get("metric_name") or rec.get("metric") or rec.get("metricName")
        joined_text = safe_join_text(rec)
        rid = rec.get("id") or rec.get("ID") or i
        row = {
            "id": rid,
            "metric_name": metric_name,
            "text": joined_text
        }
        if is_train:
           
            s = None
            for key in ["score", "label", "target", "judge_score", "llm_score"]:
                if key in rec:
                    s = rec[key]; break
           
            s = s if s is not None else rec.get("score_text")  
            if s is None:
                raise ValueError(f"No score found in train record {i}")
            row["score"] = float(s)
        rows.append(row)
    return pd.DataFrame(rows)

df_train = build_df(train, is_train=True)
df_test = build_df(test, is_train=False)


metric_to_index = {m: idx for idx, m in enumerate(metric_names)}
def get_metric_embedding(metric_name):
    
    if metric_name in metric_to_index:
        return metric_embeddings[metric_to_index[metric_name]]
    if metric_name.lower() in metric_to_index:
        return metric_embeddings[metric_to_index[metric_name.lower()]]
   
    for k in metric_to_index:
        if k.lower() == (metric_name or "").lower():
            return metric_embeddings[metric_to_index[k]]
  
    for k in metric_to_index:
        if (metric_name or "").lower() in k.lower() or k.lower() in (metric_name or "").lower():
            return metric_embeddings[metric_to_index[k]]
   
    D = metric_embeddings.shape[1]
    return np.zeros(D, dtype=np.float32)


print("Loading sentence-transformers encoder (multilingual mpnet)...")
model_name = "paraphrase-multilingual-mpnet-base-v2"  # 768-dim
encoder = SentenceTransformer(model_name)


def compute_text_embeddings(df, batch_size=64):
    texts = df["text"].astype(str).tolist()
    embs = encoder.encode(texts, batch_size=batch_size, show_progress_bar=True, convert_to_numpy=True)
    return embs

print("Encoding prompt-response text...")
train_text_emb = compute_text_embeddings(df_train)
test_text_emb = compute_text_embeddings(df_test)


print("Gathering metric embeddings for each row...")
train_metric_emb = np.vstack([get_metric_embedding(m) for m in df_train["metric_name"]])
test_metric_emb = np.vstack([get_metric_embedding(m) for m in df_test["metric_name"]])

print("Shapes:", train_text_emb.shape, train_metric_emb.shape)


def build_feature_matrix(text_emb, metric_emb, df):
  
    sims = np.array([util.cos_sim(t, m).item() for t, m in zip(text_emb, metric_emb)], dtype=np.float32).reshape(-1,1)
    absdiff = np.abs(text_emb - metric_emb)
    prod = text_emb * metric_emb
    # Text statistics
    text_len = df["text"].apply(len).to_numpy().reshape(-1,1)
    token_count = df["text"].apply(lambda s: len(s.split())).to_numpy().reshape(-1,1)
    lang = df["text"].apply(try_lang_detect).astype("category").cat.codes.to_numpy().reshape(-1,1)

    feat = np.concatenate([
        sims,
        absdiff.mean(axis=1, keepdims=True),
        absdiff.std(axis=1, keepdims=True),
        prod.mean(axis=1, keepdims=True),
        prod.std(axis=1, keepdims=True),
        text_len.astype(np.float32),
        token_count.astype(np.float32),
        lang.astype(np.float32)
    ], axis=1)
    return feat

print("Building feature matrices...")
X_train = build_feature_matrix(train_text_emb, train_metric_emb, df_train)
X_test = build_feature_matrix(test_text_emb, test_metric_emb, df_test)
y_train = df_train["score"].to_numpy().astype(np.float32)

print("Feature shapes:", X_train.shape, X_test.shape)



Loading dataset files...
Loaded 145 metric names and embeddings shape (145, 768)
Train records: 5000, Test records: 3638
Loading sentence-transformers encoder (multilingual mpnet)...
Encoding prompt-response text...


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Batches:   0%|          | 0/57 [00:00<?, ?it/s]

Gathering metric embeddings for each row...
Shapes: (5000, 768) (5000, 768)
Building feature matrices...
Feature shapes: (5000, 8) (3638, 8)


In [ ]:
# Robust CV training with LightGBM using callbacks for early stopping
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import math
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X_train), dtype=np.float32)
test_preds = np.zeros(len(X_test), dtype=np.float32)

lgb_params = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "min_data_in_leaf": 20,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.9,
    "bagging_freq": 5,
    "seed": 42,
    "verbosity": -1,
}

print("Starting CV training...")

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train)):
    print(f"\nFold {fold+1}")
    X_tr, X_val = X_train[tr_idx], X_train[val_idx]
    y_tr, y_val = y_train[tr_idx], y_train[val_idx]

  
    try:
        dtrain = lgb.Dataset(X_tr, label=y_tr)
        dvalid = lgb.Dataset(X_val, label=y_val, reference=dtrain)
      
        callbacks = [
            lgb.callback.early_stopping(stopping_rounds=100),
            lgb.callback.log_evaluation(period=100)
        ]
        model = lgb.train(
            params=lgb_params,
            train_set=dtrain,
            num_boost_round=2000,
            valid_sets=[dtrain, dvalid],
            valid_names=['train','valid'],
            callbacks=callbacks
        )

        best_iter = model.best_iteration if model.best_iteration is not None else model.num_trees()
        oof_preds[val_idx] = model.predict(X_val, num_iteration=best_iter)
        test_preds += model.predict(X_test, num_iteration=best_iter) / kf.n_splits

    except Exception as e:
     
        print("Warning: lgb.train with callbacks failed, falling back to LGBMRegressor.fit() without early stopping.")
        print("Reason:", e)
        lgbm_params_sklearn = {
            "objective": "regression",
            "learning_rate": 0.05,
            "num_leaves": 31,
            "min_child_samples": 20,
            "feature_fraction": 0.8,
            "bagging_fraction": 0.9,
            "bagging_freq": 5,  
            "random_state": 42,
            "n_estimators": 1000,   
            "verbosity": -1,
        }
        model_skl = LGBMRegressor(**lgbm_params_sklearn)
        model_skl.fit(X_tr, y_tr)  
        best_iter_skl = getattr(model_skl, "best_iteration_", model_skl.n_estimators)
        oof_preds[val_idx] = model_skl.predict(X_val, num_iteration=best_iter_skl)
        test_preds += model_skl.predict(X_test, num_iteration=best_iter_skl) / kf.n_splits

# Evaluate OOF RMSE
oof_rmse = math.sqrt(mean_squared_error(y_train, oof_preds))
print(f"\nOOF RMSE: {oof_rmse:.6f}")


Starting CV training...

Fold 1
Training until validation scores don't improve for 100 rounds
[100]	train's rmse: 0.743899	valid's rmse: 0.940151
Early stopping, best iteration is:
[26]	train's rmse: 0.851624	valid's rmse: 0.929688

Fold 2
Training until validation scores don't improve for 100 rounds
[100]	train's rmse: 0.778638	valid's rmse: 0.799399
Early stopping, best iteration is:
[97]	train's rmse: 0.783045	valid's rmse: 0.798192

Fold 3
Training until validation scores don't improve for 100 rounds
[100]	train's rmse: 0.762006	valid's rmse: 0.879065
Early stopping, best iteration is:
[29]	train's rmse: 0.862676	valid's rmse: 0.873204

Fold 4
Training until validation scores don't improve for 100 rounds
[100]	train's rmse: 0.736475	valid's rmse: 0.997871
Early stopping, best iteration is:
[91]	train's rmse: 0.744773	valid's rmse: 0.996489

Fold 5
Training until validation scores don't improve for 100 rounds
[100]	train's rmse: 0.738586	valid's rmse: 0.997823
Early stopping, best i

In [ ]:

oof_df = df_train[["id", "metric_name"]].copy()
oof_df["true_score"] = y_train
oof_df["pred_score"] = oof_preds
oof_df.to_csv(OOF_PRED_F, index=False)
print(f"Saved OOF predictions to {OOF_PRED_F}")


final_test_preds = np.clip(test_preds, 0.0, 10.0)
final_test_preds = np.round(final_test_preds * 10) / 10.0

pred_map = {
    str(iid): float(pred)
    for iid, pred in zip(df_test["id"].astype(str), final_test_preds)
}

id_col = sample_sub.columns[0]

submission_df = sample_sub.copy()
submission_df["score"] = submission_df[id_col].astype(str).map(pred_map)


missing = submission_df["score"].isna().sum()
if missing > 0:
    print(f"Warning: {missing} IDs not found in df_test. Filling with 0.0")
    submission_df["score"] = submission_df["score"].fillna(0.0)

submission_df.to_csv(OUTPUT_SUB, index=False)
print(f"Saved submission to {OUTPUT_SUB} with {len(submission_df)} rows (aligned to sample).")


Saved OOF predictions to oof_preds.csv
Saved submission to submission.csv with 3638 rows (aligned to sample).
